In [17]:
import pandas as pd

In [18]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import substring, length, col, expr

In [19]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Word Count").getOrCreate()

In [20]:

df1 = spark.read.json('web_client.json') # открываем файлы через пайспарк
df2 = spark.read.json('mobile_client.json')
df3 = spark.read.csv('parent_operator.csv')
df4 = spark.read.parquet('agg_usage.parquet')

# data_traffic

In [21]:
df4_temp=df4.select('rounded_data_volume','time_key','client_id','call_type_key','roaming_type_key','network_type')    #to create aggregate rounded_data_volume by days
df4_temp = df4_temp.withColumn("time_key",expr("substring(time_key, 2, length(time_key))")) # Я очистил time_key чтобы можно было просто сложить по месяцам
df4_temp=df4_temp.where( (df4_temp.call_type_key == 'G') | (df4_temp.call_type_key == 'Y') | (df4_temp.call_type_key == 'X'))  # чтобы call_type_key принимал X Y ил G
df4_temp=df4_temp.where( (df4_temp.roaming_type_key == 'X') | (df4_temp.roaming_type_key == 'R') | (df4_temp.roaming_type_key == 'H'))  # чтобы call_type_key принимал X Y ил G
df4_temp = df4_temp.withColumn("time_key", df4_temp["time_key"].cast(IntegerType()))

### data_4g_mb

In [22]:
data_4g_mb=df4_temp.where( (df4_temp.network_type == 'L'))

In [23]:

data_4g_mb= data_4g_mb.groupBy('client_id','time_key').sum("rounded_data_volume")  # тут я просто агрегирую по месяцам

In [24]:
data_4g_mb=data_4g_mb.withColumnRenamed("sum(rounded_data_volume)", "data_4g_mb")
data_4g_mb=data_4g_mb.withColumn("data_4g_mb", data_4g_mb.data_4g_mb/(1024*1024))

In [25]:
# data_4g_mb.show()

### data_all_mb

In [26]:

data_all_mb= df4_temp.groupBy('client_id','time_key').sum("rounded_data_volume")  # тут я просто агрегирую по месяцам
data_all_mb=data_all_mb.withColumnRenamed("sum(rounded_data_volume)", "data_all_mb")
data_all_mb=data_all_mb.select('data_all_mb','client_id')
data_all_mb=data_all_mb.withColumn('data_all_mb', data_all_mb.data_all_mb/(1024*1024))

In [27]:
# data_all_mb.show() 

### data_3g_mb

In [28]:
data_3g_mb=df4_temp.where( (df4_temp.network_type == 'G'))
data_3g_mb=data_3g_mb.groupBy('client_id','time_key').sum("rounded_data_volume")  # тут я просто агрегирую по месяцам
data_3g_mb=data_3g_mb.withColumnRenamed("sum(rounded_data_volume)", "data_3g_mb")
data_3g_mb=data_3g_mb.select('data_3g_mb','client_id')
data_3g_mb=data_3g_mb.withColumn('data_3g_mb', data_3g_mb.data_3g_mb/(1024*1024))

In [29]:
# data_3g_mb.show() 

### data_traffic

In [30]:
data_traffic=data_4g_mb.join(data_all_mb,'client_id','left')
data_traffic=data_traffic.join(data_3g_mb,'client_id','left') #соединяю все в один файл

In [31]:
# data_traffic.show()